# Install additional Python libraries

In [1]:
# !pip install quantities

# Delete existing DB

In [2]:
!rm simoc_server/db.sqlite

# Import Python libraries

In [3]:
import json
from random import randint

In [4]:
import pprint
pp = pprint.PrettyPrinter()

In [5]:
from simoc_server import app, db
from simoc_server.database.db_model import User
from simoc_server.database.seed_data import seed
from simoc_server.game_runner import GameRunnerManager, GameRunnerInitializationParams

Unknown DB_TYPE variable: "None"
Using SQLite by default


# Load Agent model

In [6]:
db.drop_all()
db.create_all()
seed.seed('agent_desc.json')

# Create new user

In [7]:
username = 'user1'
password = 'user1'

user = User.query.filter_by(username=username).first()
if not user:
    print('Creating new user')
    user = User(username=username)
    user.set_password(password)
    db.session.add(user)
    db.session.commit()

Creating new user


# Initialize the ABM

In [8]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "food_storage": [1]}, "amount": 10}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}],
    "co2_removal_SAWD": [{"connections": {"power_storage": [1], "air_storage": [1]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 0}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 1000}]},
"termination": {
    "time": {"value": 2000, "unit": "hour"}},
    "food_leaf": {"value": 10000, "unit": "kg"},
    "evacuation": {}
}

In [9]:
game_config = {"agents": {
    "human_agent": [{"connections": {"air_storage": [1], "nutrient_storage": [1], "power_storage": [1], "water_storage": [1, 2], "food_storage": [1]}, "amount": 20}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "lettuce": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    "greenhouse_medium": [{"connections": {"power_storage": [1]}, "amount": 1}],
    "solar_pv_array_mars": [{"connections": {"power_storage": [1]}, "amount": 100}],
    "multifiltration_purifier_post_treament": [{"connections": {"water_storage": [1, 2]}, "amount": 1}],
    "urine_recycling_processor_VCD": [{"connections": {"power_storage": [1], "water_storage": [1, 2]}, "amount": 1}]},
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 100}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100, "sold_wast": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000, "heat_cal": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 200}]},
"termination": [
    {"condition": "time", "value": 2, "unit": "year"},
    {"condition": "food_leaf", "value": 10000, "unit": "kg"},
    {"condition": "evacuation"}]
}

In [10]:
game_config = {"agents": {
    "solar_pv_array_mars": [{"connections": {"power_storage": [1]}, "amount": 100}]
    },
"storages": {
    "power_storage": [{"id": 1, "enrg_kwh": 0}]
    },
"termination": [
    {"condition": "time", "value": 2, "unit": "year"},
    {"condition": "food_leaf", "value": 10000, "unit": "kg"},
    {"condition": "evacuation"}]
}

In [11]:
game_config = {"agents": {
    "co2_removal_SAWD": [{"connections": {"power_storage": [1], "air_storage": [1]}, "amount": 100}],
    "cabbage": [{"connections": {"air_storage": [1], "water_storage": [1, 2], "nutrient_storage": [1],
                                 "power_storage": [1], "food_storage": [1]}, "amount": 10}],
    },
"storages": {
    "air_storage": [{"id": 1, "atmo_h2o": 100, "atmo_o2": 100, "atmo_co2": 100}],
    "water_storage": [{"id": 1, "h2o_potb": 100, "h2o_tret": 100}, {"id": 2, "h2o_wste": 100, "h2o_urin": 100}],
    "nutrient_storage": [{"id": 1, "sold_n": 100, "sold_p": 100, "sold_k": 100, "sold_wast": 100}],
    "power_storage": [{"id": 1, "enrg_kwh": 1000, "heat_cal": 1000}],
    "food_storage": [{"id": 1, "food_edbl": 200}]
    },
"termination": [
    {"condition": "time", "value": 2, "unit": "year"},
    {"condition": "food_leaf", "value": 10000, "unit": "kg"},
    {"condition": "evacuation"}]
}

In [12]:
game_runner_manager = GameRunnerManager()
game_runner_init_params = GameRunnerInitializationParams(game_config)
game_runner_manager.new_game(user, game_runner_init_params)
game_runner = game_runner_manager.get_game_runner(user)

# Run the simiulation

In [13]:
num_steps = 100
for i in range(1, num_steps + 1):
    agent_model_state = game_runner_manager.get_step(user, i)
#     if i % 100 == 0:
#         print("Step: {}".format(i))
#         pp.pprint(agent_model_state)
pp.pprint(agent_model_state)

{'agents': {'total_agent_types': {'cabbage': 10, 'co2_removal_SAWD': 100},
            'total_consumption': {'atmo_co2': {'units': 'kg',
                                               'value': '21.1038'},
                                  'enrg_kwh': {'units': 'kWh',
                                               'value': '112.9761'},
                                  'h2o_potb': {'units': 'g', 'value': '2.5529'},
                                  'sold_k': {'units': 'g', 'value': '0.0274'},
                                  'sold_n': {'units': 'g', 'value': '0.0983'},
                                  'sold_p': {'units': 'g', 'value': '0.0036'}},
            'total_production': {'atmo_o2': {'units': 'g', 'value': '2.7529'},
                                 'food_edbl': {'units': 'g',
                                               'value': '214.6250'}}},
 'hours_per_step': 1.0,
 'is_terminated': False,
 'model_stats': {'air_storage_1': {'atmo_ch4_ratio': 0,
                            

In [14]:
game_runner_manager.save_all(allow_repeat_save=False)

In [15]:
game_runner.agent_model.get_total_storages()

[{'agent_type': 'water_storage',
  'agent_id': 1,
  'currencies': [{'name': 'h2o_treated',
    'value': '0.0000',
    'capacity': 100,
    'units': 'kg'},
   {'name': 'h2o_potb', 'value': '99.8609', 'capacity': 400, 'units': 'kg'},
   {'name': 'h2o_urin', 'value': '0.0000', 'capacity': 200, 'units': 'kg'},
   {'name': 'h2o_wste', 'value': '0.0000', 'capacity': 250, 'units': 'kg'},
   {'name': 'h2o_tret', 'value': '100.0000', 'capacity': 250, 'units': 'kg'}]},
 {'agent_type': 'air_storage',
  'agent_id': 1,
  'currencies': [{'name': 'atmo_o2',
    'value': '100.3001',
    'capacity': 400,
    'units': 'kg'},
   {'name': 'atmo_co2', 'value': '0.0000', 'capacity': 200, 'units': 'kg'},
   {'name': 'atmo_n2', 'value': '0.0000', 'capacity': 400, 'units': 'kg'},
   {'name': 'atmo_ch4', 'value': '0.0000', 'capacity': 200, 'units': 'kg'},
   {'name': 'atmo_h2', 'value': '0.0000', 'capacity': 100, 'units': 'kg'},
   {'name': 'atmo_h2o', 'value': '100.0000', 'capacity': 200, 'units': 'kg'}]},
 {'

In [16]:
game_runner.agent_model.get_total_agents()

{'total_production': {'atmo_o2': {'value': '2.7529', 'units': 'g'},
  'food_edbl': {'value': '214.6250', 'units': 'g'}},
 'total_consumption': {'atmo_co2': {'value': '21.1038', 'units': 'kg'},
  'enrg_kwh': {'value': '112.9761', 'units': 'kWh'},
  'h2o_potb': {'value': '2.5529', 'units': 'g'},
  'sold_n': {'value': '0.0983', 'units': 'g'},
  'sold_p': {'value': '0.0036', 'units': 'g'},
  'sold_k': {'value': '0.0274', 'units': 'g'}},
 'total_agent_types': {'co2_removal_SAWD': 100, 'cabbage': 10}}

In [17]:
game_runner.agent_model.get_model_stats()

{'step': 109,
 'hours_per_step': 1.0,
 'is_terminated': False,
 'time': 392400.0,
 'agents': {'total_production': {'atmo_o2': {'value': '2.7529', 'units': 'g'},
   'food_edbl': {'value': '214.6250', 'units': 'g'}},
  'total_consumption': {'atmo_co2': {'value': '21.1038', 'units': 'kg'},
   'enrg_kwh': {'value': '112.9761', 'units': 'kWh'},
   'h2o_potb': {'value': '2.5529', 'units': 'g'},
   'sold_n': {'value': '0.0983', 'units': 'g'},
   'sold_p': {'value': '0.0036', 'units': 'g'},
   'sold_k': {'value': '0.0274', 'units': 'g'}},
  'total_agent_types': {'co2_removal_SAWD': 100, 'cabbage': 10}},
 'storages': [{'agent_type': 'water_storage',
   'agent_id': 1,
   'currencies': [{'name': 'h2o_treated',
     'value': '0.0000',
     'capacity': 100,
     'units': 'kg'},
    {'name': 'h2o_potb', 'value': '99.8609', 'capacity': 400, 'units': 'kg'},
    {'name': 'h2o_urin', 'value': '0.0000', 'capacity': 200, 'units': 'kg'},
    {'name': 'h2o_wste', 'value': '0.0000', 'capacity': 250, 'units':